# Compound synthesis: exploring synthetic routes and  robotic synthesis prep 
----

<font size="3">This part of the practical will involve exploring the synthesis of the follow up compounds from fragments to druglike hits. You will use retrosynthesis prediction algorithms using the Python wrapper for the IBM RXN for Chemistry API (https://github.com/rxn4chemistry/rxn4chemistry) to return potential synthetic routes and synthesis actions that can be used for robotic synthesis. You will also see how this workflow can be linked to the OpenTrons robot. A summary of tools and algorithms used by IBM RXN for Chemistry can be found at: https://rxn.res.ibm.com/index.php/ai-research-publications/.</font>

## Part 1: Create a free IBM RXN for Chemistry account and your first project
----
<font size="3">Navigate to the IBM RXN for Chemistry page: https://rxn.res.ibm.com/rxn/sign-in and click 'Login'. You can login using either your facebook or Google credentials or click 'Sign up!'. 
    
Next up, and to work with the Python wrapper for the IBM RXN Chemistry API, you need to retrieve the API key associated with your account. Click on your profile name (Top right of screen) and select 'My profile' (Fig. 1) to display your profile info and API key (Fig. 2). This is the API key you will use in this exercise.   
    
<img src="../img/IBM_API_1.png">
<strong><em>Fig. 1 - IBM RXN for Chemistry profile projects landing page</em></strong>
<img src="../img/IBM_API_2.png">
<strong><em>Fig. 2 - IBM RXN for Chemistry profile settings page</em></strong>
    

### Exercise
---

<font size="3">
    
Create your first IBM RXN for Chemistry project using your API key. 

In the cell below, we will create your first IBM RXN for Chemistry project using your API key.    

</font>

In [ ]:
from rxn4chemistry import RXN4ChemistryWrapper
import os

api_key = os.environ['IBM_API_KEY']

# Edit the next two lines to add your API key and project name information
# api_key='Insert your API key here'
project_name = 'SABS prac'

In [ ]:
# Run this code block to create your first IBM RXN for Chemistry project
rxn4chemistry_wrapper = RXN4ChemistryWrapper(api_key=api_key)
rxn4chemistry_wrapper.create_project(name=project_name)

### Sanity check - have we created a project using the API?
---

<font size="3">
    
Check if you can view the project you created by visiting: https://rxn.res.ibm.com/rxn/projects/list.    

</font>

## Part 2: Use the IBM RXN for Chemistry API to get some synthesis predictions
---

<font size="3">
Use the two functions 'getIBMRetroSyn' and 'collect_reactions' in the code block below to call the API for synthetic routes. You can find more information and about the 'rxn4chemistry_wrapper' used at: https://github.com/rxn4chemistry/rxn4chemistry/blob/master/rxn4chemistry/core.py. Check it out if you'd like to adjust the algorithms output by selecting maximum number of predicted steps, excluding reactants and setting pricing of the starting materials. 
    
You do not need to edit and only need to run the code block below.
</font>

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display
import time

# Function to get reactions from IBM API
def getIBMRetroSyn(smiles):
    results = {}
    results['status'] = None
    final_results = None

    while final_results is None:  
        try:
            response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product=smiles, max_steps=2)
            while results['status'] != 'SUCCESS': 
                time.sleep(130)
                results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(response['prediction_id'])
                final_results = results
        except Exception as e:
            print(e)
    return final_results

# Function to get all the reactions sorted into groups
def collect_reactions(tree):
    reactions = []
    if 'children' in tree and len(tree['children']):
        reactions.append(
            AllChem.ReactionFromSmarts('{}>>{}'.format(
                '.'.join([node['smiles'] for node in tree['children']]),
                tree['smiles']
            ), useSmiles=True)
        )
    for node in tree['children']:
        reactions.extend(collect_reactions(node))
    return reactions

### Moonshot time - predict the synthesis of one of the promising 3-aminopyridine-like series compounds
---

<font size="3">
You will now use the API to investigate the synthesis of a promising 3-aminopyridine-like compound that has yielded some promising IC50 assay results. Check out more about this incredible project at: https://covid.postera.ai/covid
    
<blockquote cite="https://covid.postera.ai/covid">
The COVID Moonshot is an ambitious crowdsourced initiative to accelerate the development of a COVID antiviral. We work in the open with no intellectual property constraints. This way, any scientist can view submitted drug designs and experimental data to inspire new design ideas. We use our cutting-edge machine learning tools and Folding@home's crowdsourced supercomputer to determine which drug designs to send to our partners to make and test in the lab. With each drug design tested, we get closer to our goal.    
</blockquote>

Now that we've got some functions to predict retrosynthetic pathways for a given a product, it's time to select a compound and investigate how to synthesise the target. Staying with the 3-aminopyridine-like compounds you will run the API on a compound design (Fig. 3) submitted to the Moonshot that showed promising IC50 assay results. You can view the compound at: https://covid.postera.ai/covid/submissions/b3e365b9-9ba1-48cb-bc05-132b14d906ad/1. 
    
<img src="../img/O%3DC(Nc1cncc2ccccc12)%5BC%40%40H%5D1CCOc2ccc(Cl)cc21.svg">
<strong><em>Fig. 3 - 3-aminopyridine-like Moonshot submission (MAT-POS-b3e365b9-1). SMILES: O=C(Nc1cncc2ccccc12)[C@@H]1CCOc2ccc(Cl)cc21</em></strong>

### Exercise
---
Use the 'getIBMRetroSyn' function to call the IBM RXN for Chemistry API and retrieve the retrosynthesis predictions for MAT-POS-b3e365b9-1. 

</font>

In [ ]:
results = getIBMRetroSyn('O=C(Nc1cncc2ccccc12)[C@@H]1CCOc2ccc(Cl)cc21')

### Exercise
---
<font size="3">
Execute the codeblock below that loops over the API results returned from the 'getIBMRetroSyn' function and collects the reaction pathways using the 'collect_reactions' function.

</font>

In [ ]:
for index, path in enumerate(results['retrosynthetic_paths']):
    print('Showing path {} with confidence {}:'.format(index, path['confidence']))
    for reaction in collect_reactions(path):
        display(Chem.Draw.ReactionToImage(reaction))

## Part 3: Use the API's response to generate synthetic actions 
---

<font size="3">
In this section, you will generate synthesis actions for the the most promising pathway. You can think of actions as synthetic actions broken down into steps such as: addition of a compound, stirring, removing solvent (concentration) etc. 
</font>

### Exercise
---
<font size="3">
'results['retrosynthetic_paths']' contains the four synthetic pathways in decreasing confidence as shown above in a list. Run the code block to create a 'pathway_response' that will be a JSON object of all the actions needed to complete the synthesis.   
</font>

In [ ]:
path = results['retrosynthetic_paths'][0] 
pathway_response = rxn4chemistry_wrapper.create_synthesis_from_sequence(sequence_id=path['sequenceId'])
new_synthesis_id = pathway_response['synthesis_id']

<font size="3">
At this point, you will have a JSON response 'pathway_response' of a single pathway with the highest synthesis confidence. The IBB RXN for Chemistry API can do something very neat and predict the synthetic steps or actions (Think add, make solution, concentrate etc) needed to execute the synthesis. Note the 'name' of the synthesis actions from the output below. Can you think of others?    
</font>

In [ ]:
# get the entire list of actions for the entire synthesis, as well as a tree representation
synthesis_tree, reactions, actions = rxn4chemistry_wrapper.get_synthesis_plan(synthesis_id=new_synthesis_id)

for i, action in enumerate(actions, start=1):
    print(i, action)

<font size="3">
That's a lot of actions and synthesis work! For brevity, you will filter these actions to only include 'add' actions and leave out the addition of solvents. The final list of compounds to add will be stored in 'compounds_to_add' 
</font>

In [ ]:
ignore_compounds = ['water', 'THF', 'methanol', 'ethyl acetate', 'SLN']

compounds_to_add = []

for i, action in enumerate(actions, start=1):
        if action['name'] == 'add':
            compound_added = action['content']['material']['value']
            if compound_added not in ignore_compounds:
                compounds_to_add.append(compound_added)

In [ ]:
compounds_to_add

<font size="3">
OK, so we've got some compounds to add but some are in SMILES format and the others have a chemical name. SMILES are extremely useful for machines but chemist's feel more comfortable using names and especially images. In the next section, you will use query the NCI Cactus Resolver tool (https://cactus.nci.nih.gov) to convert chemical names into SMILES which you'll then use to generate some images. The SMILES will be stored as a list in 'smiles'   
</font>

In [ ]:
from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(name):
    try:
        name_converted = quote(name)
        url= 'https://cactus.nci.nih.gov/chemical/structure/' + name_converted + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        smiles = ans.split(' ')[0]
        return smiles
    except:
        return 'Did not work'

In [ ]:
identifiers  = compounds_to_add
# SMILES stored in smiles list
smiles = []
for ids in identifiers :
    smiles.append(CIRconvert(ids))

<font size="3">
Now that you've got some SMILES, you can generate some rdkit mol objects and draw some images. These are the compounds that you will use in the OpenTrons robotic synthesis procedure.    
</font>

In [ ]:
mols = [Chem.MolFromSmiles(smi) for smi in smiles]

In [ ]:
legends = [smi for smi in smiles]
Chem.Draw.MolsToGridImage(mols, molsPerRow=4, legends=legends)

 ## Part 4: Performing a robotic synthesis on the OpenTrons 
---

<font size="3">
In this section, all going well, you will observe a life demo of the OpenTrons robot in action. The code being run on the robot is shown in the next code block. The OpenTrons deck's setup is shown in Fig. 4 along with the location of the reactants, labware names/position and the colors used in the demo. 
    
<img src="../img/OT_deck_setup.png">
<strong><em>Fig. 4 - Labware, trough and reaction plate setup on OpenTrons deck</em></strong>
</font>

In [ ]:
# from opentrons import protocol_api

metadata = {
    'protocolName': 'Sudo synthesis of MAT-POS-b3e365b9-1',
    'author': 'SABS/DLS practical',
    'description': 'Demo of OpenTrons robotics for synthesis of MAT-POS-b3e365b9-1.',
    'apiLevel': '2.6'
}

def run(protocol: protocol_api.ProtocolContext):

    # Load the labware and assign the rack position
    reactants_trough = protocol.load_labware('4welltrough_4_reservoir_60000ul', 4)
    reaction_plate = protocol.load_labware('plateone_96_wellplate_2500ul', 5)
    
    # Load tipracks and assign a rack position
    tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 7)

    # Load the pipettes and assign the pipettes position
    p300 = protocol.load_instrument('p300_multi_gen2', 'left', tip_racks=[tiprack_1])   
        
    # Lets add the ester and base for the dehydration reaction
    # Pick up tips and add the ester to the reaction plate
    p300.transfer(200, reactants_trough['A1'].bottom(2), reaction_plate['A1'].top(-5), air_gap = 15)
    
    # Pick up tips and add the base to the reaction plate. Use 'mix_after' to mix the two reactants together
    p300.transfer(200, reactants_trough['A2'].bottom(2), reaction_plate['A1'].bottom(2), air_gap = 15, mix_after=(3,100))

    # Pause robot for reaction to complete
    protocol.pause('Press resume after 1.5 h')

    # With the dehydration reaction complete, add the amine for the final reaction towards 
    # the 3-aminopyridine-like MAT-POS-b3e365b9-1 
    # Pick up tips and add the amine to the reaction mixture
    p300.transfer(200, reactants_trough['A3'].bottom(2), reaction_plate['A1'].bottom(2), air_gap = 15, mix_after=(3,100))